In [1]:
import pandas as pd

volume = pd.read_csv("DIJA_REDDIT_Vol_Hig_Low.csv", engine="python", sep=",", error_bad_lines=False)

In [2]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /Users/Damian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Damian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


volume['cleanText']=volume['Title'].map(lambda s:preprocess(s))

In [4]:
volume.head()

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Adj Close,returns,Volatiluity,Title,cleanText,compound,neg,neu,pos
0,259,40304,10868.12012,10879.75977,9869.620117,10520.32031,459890000,10520.32031,-0.031887,3.004381,b'How China and India Sabotaged the UN Climate...,china india sabotaged climate summit secret re...,-0.9935,0.247,0.629,0.124
1,577,40764,10810.91016,11244.00977,10604.070310,11239.76953,431410000,11239.76953,0.039771,2.806139,This is serious Reddit - London riots spreadin...,serious reddit london riots spreading looting ...,-0.9984,0.339,0.596,0.065
2,1593,42240,16459.75000,16459.75000,15370.330080,15871.34961,293920000,15871.34961,-0.035748,3.037195,Twitter has forced 30 websites that archive po...,twitter forced websites archive politician del...,-0.9929,0.239,0.675,0.087
3,1594,42241,15882.26953,16312.94043,15651.240230,15666.44043,213220000,15666.44043,-0.012911,2.820661,The first UN privacy chief has said the world ...,first privacy chief said world needs geneva co...,-0.9764,0.212,0.647,0.141
4,1595,42242,15676.25977,16303.75000,15676.259770,16285.50977,208420000,16285.50977,0.039516,2.797607,Switzerland has completed the construction of ...,switzerland completed construction long gottha...,-0.9859,0.166,0.750,0.084


In [5]:
del volume['compound']
del volume['neg']
del volume['neu']
del volume['pos']

In [6]:
volume.head()

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Adj Close,returns,Volatiluity,Title,cleanText
0,259,40304,10868.12012,10879.75977,9869.620117,10520.32031,459890000,10520.32031,-0.031887,3.004381,b'How China and India Sabotaged the UN Climate...,china india sabotaged climate summit secret re...
1,577,40764,10810.91016,11244.00977,10604.070310,11239.76953,431410000,11239.76953,0.039771,2.806139,This is serious Reddit - London riots spreadin...,serious reddit london riots spreading looting ...
2,1593,42240,16459.75000,16459.75000,15370.330080,15871.34961,293920000,15871.34961,-0.035748,3.037195,Twitter has forced 30 websites that archive po...,twitter forced websites archive politician del...
3,1594,42241,15882.26953,16312.94043,15651.240230,15666.44043,213220000,15666.44043,-0.012911,2.820661,The first UN privacy chief has said the world ...,first privacy chief said world needs geneva co...
4,1595,42242,15676.25977,16303.75000,15676.259770,16285.50977,208420000,16285.50977,0.039516,2.797607,Switzerland has completed the construction of ...,switzerland completed construction long gottha...


In [7]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [8]:
pip install vaderSentiment 

Note: you may need to restart the kernel to use updated packages.


In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [10]:
preprocess = []
analyzer = SentimentIntensityAnalyzer()
for clean in volume['cleanText']:
    vs = analyzer.polarity_scores(clean)
    preprocess.append(vs)

In [11]:
#positive sentiment: compound score >= 0.05
#neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
#negative sentiment: compound score <= -0.05

In [12]:
df3 = pd.DataFrame(preprocess)

In [13]:
complete_volume = pd.concat([volume,df3], axis=1)

In [14]:
complete_volume

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Adj Close,returns,Volatiluity,Title,cleanText,compound,neg,neu,pos
0,259,40304,10868.12012,10879.75977,9869.620117,10520.32031,459890000,10520.32031,-0.031887,3.004381,b'How China and India Sabotaged the UN Climate...,china india sabotaged climate summit secret re...,-0.9935,0.247,0.629,0.124
1,577,40764,10810.91016,11244.00977,10604.070310,11239.76953,431410000,11239.76953,0.039771,2.806139,This is serious Reddit - London riots spreadin...,serious reddit london riots spreading looting ...,-0.9984,0.339,0.596,0.065
2,1593,42240,16459.75000,16459.75000,15370.330080,15871.34961,293920000,15871.34961,-0.035748,3.037195,Twitter has forced 30 websites that archive po...,twitter forced websites archive politician del...,-0.9929,0.239,0.675,0.087
3,1594,42241,15882.26953,16312.94043,15651.240230,15666.44043,213220000,15666.44043,-0.012911,2.820661,The first UN privacy chief has said the world ...,first privacy chief said world needs geneva co...,-0.9764,0.212,0.647,0.141
4,1595,42242,15676.25977,16303.75000,15676.259770,16285.50977,208420000,16285.50977,0.039516,2.797607,Switzerland has completed the construction of ...,switzerland completed construction long gottha...,-0.9859,0.166,0.750,0.084


In [16]:
complete_volume.to_csv('top_5_volume.csv')